In [4]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.preprocessing import Normalizer


In [5]:
traindata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_train.csv', index_col=None)
testdata = pd.read_csv('C:/Users/Aditya Kyatham/Documents/BE FINAL/nsl-kdd/NSL_KDD-master/Small Training Set_5class_test.csv', index_col=None)

traindata.head()

,duration,"""protocol_type""","""service""",flag,"""src_bytes""","""dst_bytes""",land,wrong_fragment,urgent,"""hot""",...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_diff_srv_rate.1,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,S1,278,7958,0,0,0,0,...,255,1.00,0.00,0.33,0.03,0.33,0.0,0.0,0.0,normal
1,0,tcp,http,SF,241,851,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.00,0.0,0.0,0.0,normal
2,0,udp,private,SF,105,146,0,0,0,0,...,239,0.94,0.01,0.01,0.00,0.00,0.0,0.0,0.0,normal
3,0,tcp,http,SF,367,5199,0,0,0,0,...,255,1.00,0.00,0.01,0.05,0.00,0.0,0.0,0.0,normal
4,21263,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.66,1.00,0.00,0.00,0.0,1.0,1.0,probe


In [7]:
from sklearn.preprocessing import LabelEncoder
encodings = dict()
for c in traindata.columns:
    #print df[c].dtype
    if traindata[c].dtype == "object":
        encodings[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings[c]
        traindata[c] = encodings[c].fit_transform(traindata[c])

In [8]:
encodings1 = dict()
for c in testdata.columns:
    #print df[c].dtype
    if testdata[c].dtype == "object":
        encodings1[c] = LabelEncoder() #to give numerical label to char type labels.
        encodings1[c]
        testdata[c] = encodings1[c].fit_transform(testdata[c])

In [9]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]


scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)


In [13]:
from time import time

from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3,loss='squared_hinge',alpha = 1)
t0 = time()
clf.fit(traindata,trainlabel)
tt = time() - t0
print ("Classifier trained in {} seconds.".format(round(tt, 3)))
clf

Classifier trained in 0.141 seconds.


SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='squared_hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
#Predictions on the test data
t0 = time()
pred = clf.predict(testdata)
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

Predicted in 0.001 seconds


In [15]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(testlabel, pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(testlabel, pred))
print('Report : ')
print(classification_report(testlabel, pred))


Confusion Matrix :
[[63  1  0  0  0]
 [ 9 99  0  0  0]
 [15 11  0  0  0]
 [ 0  2  0  0  0]
 [ 0  3  0  0  0]]
Accuracy Score : 0.7980295566502463
Report : 
              precision    recall  f1-score   support

           0       0.72      0.98      0.83        64
           1       0.85      0.92      0.88       108
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         3

    accuracy                           0.80       203
   macro avg       0.32      0.38      0.34       203
weighted avg       0.68      0.80      0.73       203



C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
t0 = time()
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
predicted = cross_val_predict(clf, traindata,trainlabel, cv=skf)
print('Accuracy Score :',accuracy_score(trainlabel, predicted))
print('Report : ')
print(classification_report(trainlabel, predicted))
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy Score : 0.8353960396039604
Report : 
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       316
           1       0.83      0.94      0.88       408
           2       0.00      0.00      0.00        65
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         8

    accuracy                           0.84       808
   macro avg       0.33      0.37      0.35       808
weighted avg       0.75      0.84      0.79       808

Predicted in 0.294 seconds


In [19]:
#Predictions on the test data
t0 = time()
pred = skf.predict(testdata)
tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

AttributeError: 'StratifiedKFold' object has no attribute 'predict'

In [18]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
 
results = confusion_matrix(testlabel, pred) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(testlabel, pred))
print('Report : ')
print(classification_report(testlabel, pred))


Confusion Matrix :
[[63  1  0  0  0]
 [ 9 99  0  0  0]
 [15 11  0  0  0]
 [ 0  2  0  0  0]
 [ 0  3  0  0  0]]
Accuracy Score : 0.7980295566502463
Report : 
              precision    recall  f1-score   support

           0       0.72      0.98      0.83        64
           1       0.85      0.92      0.88       108
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         3

    accuracy                           0.80       203
   macro avg       0.32      0.38      0.34       203
weighted avg       0.68      0.80      0.73       203



C:\Users\Aditya Kyatham\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
